In [2]:
%cd /workspace/LLaMA-Factory
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspace/LLaMA-Factory


In [4]:
test_converse = '''고객: 안녕하세요, 제 이름은 김지수입니다. 얼마 전에 주문한 제품이 아직 도착하지 않았습니다. 제 주문번호는 SH12345이고, 저희 집 주소는 서울시 강남구 테헤란로 123-45입니다. 확인 부탁드립니다.

상담원: 안녕하세요, 김지수 고객님. 저희 쇼핑몰을 이용해 주셔서 감사합니다. 고객님의 주문 상황을 확인하고 있습니다. 잠시만 기다려 주시겠어요?

상담원: 확인 결과, 고객님의 주문은 택배사의 물류 지연으로 인해 배송이 지연되고 있습니다. 통상적으로 2~3일 내에는 배송될 예정입니다. 불편을 드려 죄송합니다.

고객: 지연 사유를 알려주셔서 감사합니다. 배송 예정일을 알 수 있을까요?

상담원: 네, 현재로서는 목요일까지는 도착할 것으로 예상됩니다. 도착하면 고객님께 연락드리겠습니다. 연락 받으실 전화번호가 010-1234-5678이 맞으신가요?

고객: 네, 맞습니다. 그 전화번호로 알려주세요.

상담원: 확인했습니다. 만약 배송 관련 추가 문의사항이 있으시면 언제든지 연락주세요. 혹시 환불이 필요하시다면 연결된 계좌로 환불 처리해 드릴 수 있습니다. 계좌번호는 하나은행 123-456-789012로 확인됩니다.

고객: 네, 감사합니다. 그럼 기다리겠습니다.

상담원: 감사합니다, 김지수 고객님. 불편을 드려 다시 한번 사과드립니다. 좋은 하루 보내세요!'''

In [11]:
org_tokenizer = AutoTokenizer.from_pretrained('allganize/Llama-3-Alpha-Ko-8B-Instruct')
org_model = AutoModelForCausalLM.from_pretrained('allganize/Llama-3-Alpha-Ko-8B-Instruct')
org_model = torch.nn.DataParallel(org_model).cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
device = "cuda:0"

messages = [
    {"role": "system", "content": "당신은 개인정보를 감춰주는 로봇입니다.\n\n## 지시 사항 ##\n1.주어진 대화에서 사람이름을 [PERSON1], [PERSON2] 등으로 등장 순서에 따라 대체하고, 동일한 이름이 반복될 경우 같은 대치어를 사용합니다.\n2.연락처, 이메일, 주소 , 계좌번호도 각각 [CONTACT1], [CONTACT2] 등, [EMAIL1],[EMAIL2] 등, [ADDRESS1],[ADDRESS2]등 , [ACCOUNT1], [ACCOUNT2] 등 으로 대치하고 동일한 정보가 반복되는 경우에는 같은 대치어를 사용합니다.\n3.대치어를 작성할때 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 \n4.위 규칙은 대화 전체에 걸쳐 일관되게 적용합니다. \n당신이 개인정보를 감출 대화내역입니다."},
    {"role": "user", "content": f"입력: {test_converse}"}
]

org_encodeds = org_tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
org_model_inputs = org_encodeds.to(device)

In [13]:
input_text = org_tokenizer.decode(org_model_inputs[0])
inputs = org_tokenizer(input_text, return_tensors="pt")
eos_token_id = org_tokenizer.convert_tokens_to_ids("<|eot_id|>")

with torch.no_grad():
    outputs = org_model.module.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, eos_token_id=eos_token_id)
    print(org_tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 개인정보를 감춰주는 로봇입니다.

## 지시 사항 ##
1.주어진 대화에서 사람이름을 [PERSON1], [PERSON2] 등으로 등장 순서에 따라 대체하고, 동일한 이름이 반복될 경우 같은 대치어를 사용합니다.
2.연락처, 이메일, 주소, 계좌번호도 각각 [CONTACT1], [CONTACT2] 등, [EMAIL1],[EMAIL2] 등, [ADDRESS1],[ADDRESS2]등, [ACCOUNT1], [ACCOUNT2] 등 으로 대치하고 동일한 정보가 반복되는 경우에는 같은 대치어를 사용합니다.
3.대치어를 작성할때 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 
4.위 규칙은 대화 전체에 걸쳐 일관되게 적용합니다. 
당신이 개인정보를 감출 대화내역입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력: 고객: 안녕하세요, 제 이름은 김지수입니다. 얼마 전에 주문한 제품이 아직 도착하지 않았습니다. 제 주문번호는 SH12345이고, 저희 집 주소는 서울시 강남구 테헤란로 123-45입니다. 확인 부탁드립니다.

상담원: 안녕하세요, 김지수 고객님. 저희 쇼핑몰을 이용해 주셔서 감사합니다. 고객님의 주문 상황을 확인하고 있습니다. 잠시만 기다려 주시겠어요?

상담원: 확인 결과, 고객님의 주문은 택배사의 물류 지연으로 인해 배송이 지연되고 있습니다. 통상적으로 2~3일 내에는 배송될 예정입니다. 불편을 드려 죄송합니다.

고객: 지연 사유를 알려주셔서 감사합니다. 배송 예정일을 알 수 있을까요?

상담원: 네, 현재로서는 목요일까지는 도착할 것으로 예상됩니다. 도착하면 고객님께 연락드리겠습니다. 연락 받으실 전화번호가 010-1234-5678이 맞으신가요?

고객: 네, 맞습니다. 그 전화번호로 알려주세요.

상담원: 확인했습니다. 만약 배송 관련 

---------------

In [5]:
tokenizer = AutoTokenizer.from_pretrained('./deidentification/model')
model = AutoModelForCausalLM.from_pretrained('./deidentification/model')
model = torch.nn.DataParallel(model).cuda()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
device = "cuda:0"

messages = [
    {"role": "system", "content": "당신은 개인정보를 감춰주는 로봇입니다.\n\n## 지시 사항 ##\n1.주어진 대화에서 사람이름을 [PERSON1], [PERSON2] 등으로 등장 순서에 따라 대체하고, 동일한 이름이 반복될 경우 같은 대치어를 사용합니다.\n2.연락처, 이메일, 주소 , 계좌번호도 각각 [CONTACT1], [CONTACT2] 등, [EMAIL1],[EMAIL2] 등, [ADDRESS1],[ADDRESS2]등 , [ACCOUNT1], [ACCOUNT2] 등 으로 대치하고 동일한 정보가 반복되는 경우에는 같은 대치어를 사용합니다.\n3.대치어를 작성할때 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 \n4.위 규칙은 대화 전체에 걸쳐 일관되게 적용합니다. \n당신이 개인정보를 감출 대화내역입니다."},
    {"role": "user", "content": f"입력: {test_converse}"}
]

encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
model_inputs = encodeds.to(device)

In [7]:
model_inputs

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 111097,
         105922,  18918, 103185,  54596,    108, 119087,  72115, 103850,    229,
          80052,    382,    567,  67890,  30426, 126840,  48826,     16,     13,
          55430, 125441,  62060,  57390,  57575, 117559,  64254,  18359,    510,
          94283,     16,   1145,    510,  94283,     17,     60,  78102,  43139,
         126912, 106248,  27796,  19954, 106725,  62060,  50643, 101360,     11,
         121731,  24486,  87134,  13094,  64857,  98934, 113191,  50152, 105718,
          62060,  60798,  32179,  18918,  41820,  61938,    627,     17,     13,
         101347, 105316, 102657,     11,  23955,  85767,  33177,     11,  56773,
          44690,   1174,  95303, 113002,  73148,  49085, 127141,    510,  74471,
             16,   1145,    510,  74471,     17,     60,  78102,     11,    510,
          37004,     16,  15304,  37004,     17,     60,  78102,     11,    510,
          62105,     16,  15

In [8]:
input_text = tokenizer.decode(model_inputs[0])

In [9]:

inputs = tokenizer(input_text, return_tensors="pt")
eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 개인정보를 감춰주는 로봇입니다.

## 지시 사항 ##
1.주어진 대화에서 사람이름을 [PERSON1], [PERSON2] 등으로 등장 순서에 따라 대체하고, 동일한 이름이 반복될 경우 같은 대치어를 사용합니다.
2.연락처, 이메일, 주소, 계좌번호도 각각 [CONTACT1], [CONTACT2] 등, [EMAIL1],[EMAIL2] 등, [ADDRESS1],[ADDRESS2]등, [ACCOUNT1], [ACCOUNT2] 등 으로 대치하고 동일한 정보가 반복되는 경우에는 같은 대치어를 사용합니다.
3.대치어를 작성할때 글머리 기호나, 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오 
4.위 규칙은 대화 전체에 걸쳐 일관되게 적용합니다. 
당신이 개인정보를 감출 대화내역입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력: 고객: 안녕하세요, 제 이름은 김지수입니다. 얼마 전에 주문한 제품이 아직 도착하지 않았습니다. 제 주문번호는 SH12345이고, 저희 집 주소는 서울시 강남구 테헤란로 123-45입니다. 확인 부탁드립니다.

상담원: 안녕하세요, 김지수 고객님. 저희 쇼핑몰을 이용해 주셔서 감사합니다. 고객님의 주문 상황을 확인하고 있습니다. 잠시만 기다려 주시겠어요?

상담원: 확인 결과, 고객님의 주문은 택배사의 물류 지연으로 인해 배송이 지연되고 있습니다. 통상적으로 2~3일 내에는 배송될 예정입니다. 불편을 드려 죄송합니다.

고객: 지연 사유를 알려주셔서 감사합니다. 배송 예정일을 알 수 있을까요?

상담원: 네, 현재로서는 목요일까지는 도착할 것으로 예상됩니다. 도착하면 고객님께 연락드리겠습니다. 연락 받으실 전화번호가 010-1234-5678이 맞으신가요?

고객: 네, 맞습니다. 그 전화번호로 알려주세요.

상담원: 확인했습니다. 만약 배송 관련 

In [19]:
from huggingface_hub import HfApi
api = HfApi()
username = "irene93"

MODEL_NAME = 'Llama-3-deidentifier'
api.create_repo(
    token=token,
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token=token ,
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="./deidentification/model",
)

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/irene93/Llama-3-deidentifier/commit/a55851aeaf03e187dc3527e776afd3498e6bf3a8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a55851aeaf03e187dc3527e776afd3498e6bf3a8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/irene93/Llama-3-deidentifier', endpoint='https://huggingface.co', repo_type='model', repo_id='irene93/Llama-3-deidentifier'), pr_revision=None, pr_num=None)